In [47]:
#import files
import os
import json
import pprint
import pandas as pd
import numpy as np
import requests

import json
import geojson
import folium

import sklearn

from folium.features import GeoJsonTooltip
from geopy.geocoders import Nominatim

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime


In [48]:
#all are from consumerfinance.gov from 1.1.20XX to 12.31.20YY being load as debt_collect_XXYY 

#Product is Debt Collection, Focus on Untimely Response

# loading the json
with open("debt_complaints_untimely_response 2014-2022.json") as debt_collect:
    debt_collect = json.load(debt_collect) 

In [ ]:
# pprint.pprint(debt_collect[0])
# {'_id': '3262709',
#  '_index': 'complaint-public-v1',
#  '_score': None,
#  '_source': {'company': 'Capital Accounts, LLC',
#              'company_public_response': 'Company believes it acted '
#                                         'appropriately as authorized by '
#                                         'contract or law',
#              'company_response': 'Closed with explanation',
#              'complaint_id': '3262709',
#              'complaint_what_happened': 'Please be advised this is my SECOND '
#                                         'WRITTEN REQUEST asking to remove the '
#                                         'account listed below that remain on '
#                                         'my credit report in violation of 15 '
#                                         'U.S.C. ss 1681. Reporting inaccurate '
#                                         'and unsubstantiated information to a '
#                                         'credit reporting agency may '
#                                         'constitute fraud under federal law. '
#                                         'This company has ignored my request '
#                                         'to provide me with the documents that '
#                                         'they have in their files that they '
#                                         'used to validate the disputed account '
#                                         'which means that they have NOT '
#                                         'validated or can not validate this '
#                                         'account and under section 611 ( 5 ) ( '
#                                         'A ) of the FCRA- they are required to '
#                                         '" promptly DELETE information which '
#                                         "can not validate or verify. '' The "
#                                         'law is very clear as to the Civil '
#                                         'liability and remedy available to me '
#                                         'for " negligent noncompliance \'\' ( '
#                                         'Section 617 ) if you fail to comply '
#                                         'with this Federal Law. I am a '
#                                         'litigious consumer and following '
#                                         'accounts be validated immediately. A '
#                                         'copy of this complaint as well as '
#                                         'copies of the previously written '
#                                         'letter sent to you by certified mail '
#                                         'with proof of delivery will also '
#                                         'become part of a formal complaint to '
#                                         'the Better Business Bureau. They will '
#                                         'also be submitted as evidence in '
#                                         'pending litigation provided you fail '
#                                         'to comply with this complaint. Once '
#                                         'again, the law is very clear as to '
#                                         'the civil liability and the remedy '
#                                         'available to me ( Section 616 & 617 ) '
#                                         'if you fail to comply with Federal '
#                                         'Law. I intend on pursuing litigation '
#                                         'in this matter to enforce my rights '
#                                         'under the FCRA. I do not have a '
#                                         'signed contract nor any contract with '
#                                         'CAPITAL ACCOUNTS LLC. I have never '
#                                         'done business with this company. I am '
#                                         'not liable for collection from this '
#                                         'debt from CAPITAL ACCOUNTS LLC. I am '
#                                         'not responsible for this debt with '
#                                         'CAPITAL ACCOUNTS LLC. Also per the '
#                                         'State of Florida Office of Financial '
#                                         'Regulatory you must be licensed and '
#                                         'bonded in the state of Florida to '
#                                         'collect.',
#              'consumer_consent_provided': 'Consent provided',
#              'consumer_disputed': 'N/A',
#              'date_received': '2019-06-03T12:00:00-05:00',
#              'date_sent_to_company': '2019-06-03T12:00:00-05:00',
#              'issue': 'Communication tactics',
#              'product': 'Debt collection',
#              'state': 'FL',
#              'sub_issue': 'Used obscene, profane, or other abusive language',
#              'sub_product': 'I do not know',
#              'submitted_via': 'Web',
#              'tags': None,
#              'timely': 'No',
#              'zip_code': 'XXXXX'},
#  '_type': '_doc',
#  'sort': [60]}



In [49]:
issue = []
sub_issue =[]
company = []
company_response = []
complaint_long_desc = []
consumer_disputed =[]
state = []
year=[]
month =[]
# zip_code = []
  
for x in range(len(debt_collect)): 
    company.append(debt_collect[x]['_source']['company'])
    issue.append(debt_collect[x]['_source']['issue'])
    sub_issue.append(debt_collect[x]['_source']['sub_issue'])
    company_response.append(debt_collect[x]['_source']['company_response'])
    consumer_disputed.append(debt_collect[x]['_source']['consumer_disputed'])
    state.append(debt_collect[x]['_source']['state'])
    date_time_obj = datetime.fromisoformat(debt_collect[x]['_source']['date_received'])
    year.append(date_time_obj.year)
    month.append(date_time_obj.month)

#     zip_code.append(debt_collect[x]['_source']['zip_code'])

debt_df = pd.DataFrame({'issue': issue, "sub_issue": sub_issue, "company":company, "company_response":company_response,"state":state, 'year':year, 'month':month })
debt_df

,issue,sub_issue,company,company_response,state,year,month
0,Communication tactics,"Used obscene, profane, or other abusive language","Capital Accounts, LLC",Closed with explanation,FL,2019,6
1,Written notification about debt,Didn't receive enough information to verify debt,"Crown Asset Management, LLC, Duluth, GA Branch",Closed with explanation,NY,2019,12
2,Attempts to collect debt not owed,Debt was already discharged in bankruptcy and ...,"Med-1 Solutions, LLC",Closed with explanation,IN,2018,9
3,Took or threatened to take negative or legal a...,Threatened to sue you for very old debt,"Merchants Credit Bureau, Inc.",Closed with explanation,GA,2019,2
4,False statements or representation,Attempted to collect wrong amount,"Bull City Financial Solutions, Inc",Closed with explanation,CT,2018,7
...,...,...,...,...,...,...,...
21534,Communication tactics,Frequent or repeated calls,"CREDIT BUREAU OF NAPA COUNTY, INC.",Closed with explanation,NJ,2015,12
21535,Attempts to collect debt not owed,Debt is not yours,"Best Account Receivables Management Solutions,...",Untimely response,CA,2022,8
21536,Attempts to collect debt not owed,Debt is not yours,"Commonwealth Financial Systems, Inc.",Closed with explanation,MD,2022,11
21537,Improper contact or sharing of info,Talked to a third party about my debt,"Sarasota CCM, Inc.",Closed,TX,2015,3


In [ ]:
#More graph to examinine data

#color palete
sns.set_palette('tab10')

# Bar Plot 1
plt.figure(figsize=(12, 4))
sns.countplot(y='issue', data=debt_df, )
    
plt.title('Types of Issue')
plt.show()

# Bar Plot 2
plt.figure(figsize=(12, 12))
sns.countplot(y='sub_issue', data=debt_df)
plt.title('Types of sub_issue')
plt.show()


In [110]:
# line plot with state
counts_per_yr = debt_df.groupby(['year', 'month']).count().reset_index()
#counts_per_yr
# Create bar plot using Seaborn
sns.set_theme(style="whitegrid")

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=counts_per_yr, kind="bar",
    x="year", y="issue", hue="month",
    errorbar="sd", palette="dark", alpha=.6, height=60, aspect =5,
)
g.despine(left=True)
g.set_axis_labels("", "Number of Issue Per Month/Year")
g.legend.set_title("Number of Issue Per Month/Year")


In [114]:
counts_per_yr.pivot("year")


TypeError: pivot() missing 1 required argument: 'columns'